In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

SHEET_NAMES = ["Pre & Post", "Weekly", "Daily"]

In [3]:
df = pd.read_csv("data/class_data.csv", encoding = 'latin-1')
df_interested = df[df["pre_studyinterest"] == "Yes"]

In [13]:
def find_percentage(df):
    nrows = df.shape[0]
    df_percentage = pd.DataFrame(df.count()).reset_index()
    df_percentage.columns = ["var_name", "count"]    
    df_percentage["null_count"] = nrows - df_percentage["count"]
    df_percentage["null_percentage"] = df_percentage["null_count"] / nrows
    return df_percentage


# 3 possibilities: varname, pre/post_varname, varname"dnum", varname"wnum"
def find_name_stem(df, sheet_name):
    codebook = pd.read_excel("data/codebook.xlsx", sheet_name=sheet_name)
    construct_
    
    

In [124]:
# construct possible varnames for each Name Stem in its Construct group
# flag in ["pre", "post", 'd', 'w']
def build_counstruct_variable_dict(codebook, flag):
    appending = [c for c in list(codebook.columns)[1:list(codebook.columns).index("Construct")]]
    if "pre" in appending:
        construct_var_dict = \
            codebook.groupby("Construct").apply(
                lambda d: {i: [a + "_" + i for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                           for i in d["Name Stem"]}).to_dict()
            
    else:
        construct_var_dict = \
            codebook.groupby("Construct").apply(
                lambda d: {i: [i + flag + str(a) for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                           for i in d["Name Stem"]}).to_dict()
    return construct_var_dict

# Verify possuble varnames by checking in bio class variables set
def find_construct_variable(df, codebook, flag):
    construct_var_dict = build_counstruct_variable_dict(codebook, flag)
    return {construct: {var_root: [var for var in construct_var_dict[construct][var_root] + [var_root] if var in df.columns] \
                          for var_root in construct_var_dict[construct]} for construct in construct_var_dict}

def print_non_exist_variable(df, to_ret):
    print("Name Stems that are not in df:  ")
    for construct in to_ret:
        for var_root in to_ret[construct]:
            if len(to_ret[construct][var_root]) == 0:
                print([construct, var_root])
    print('\n')

def build_var_null_percentage_df(df, sheet_name, verbal = False, overwrite = True, student_interested = False):
    def construct_percentage_df(construct_var_dict):
        df_percentage_master = pd.DataFrame()
        for construct in construct_var_dict:
            found_var = [var for var_root in construct_var_dict[construct] for var in construct_var_dict[construct][var_root]]
            df_construct_sub = df[found_var]
            to_view = pd.DataFrame(df_construct_sub.count()).reset_index()
            to_view.columns = ["var", "count"]
            to_view["null_count"] = df_construct_sub.shape[0] - to_view["count"]
            to_view["null_percentage"] = to_view["null_count"] / df_construct_sub.shape[0]
            to_view["Name Stem"] = [var_root for var_root in construct_var_dict[construct] for var in construct_var_dict[construct][var_root]]
            to_view["Construct"] = construct
            df_percentage_master = pd.concat([df_percentage_master, to_view], axis = 0)
        return df_percentage_master
    
    df_codebook =  pd.read_excel("data/codebook.xlsx", sheet_name=sheet_name)
    construct_var_dict = find_construct_variable(df, df_codebook, sheet_name[0].lower())
    df_percentage = construct_percentage_df(construct_var_dict)
    df_percentage["flag"] = [i.replace(j, '').replace('_', '') if i.replace(j, '').replace('_', '') else "original" 
                       for i, j in zip(df_percentage["var"], df_percentage["Name Stem"])]
    df_percentage = pd.DataFrame(df_percentage.pivot(index='Name Stem', columns="flag", values=["count", "null_count", "null_percentage"]))
    df_percentage.columns =['_'.join(col[::-1]).strip() for col in df_percentage.columns.values]
    codebook_vars = ["Name Stem", "Construct", "Item", "Response Values", "Label"]
    df_percentage = pd.DataFrame(df_percentage).reset_index().merge(df_codebook[codebook_vars], 
                                                                   left_on = "Name Stem", right_on = "Name Stem", how = "left")
    df_percentage = df_percentage[codebook_vars + [c for c in df_percentage.columns if c not in codebook_vars]]
    if verbal:
        print(sheet_name, '\n\n')
        print_non_exist_variable(df, construct_var_dict)
        display(df_percentage.head())
    
    if overwrite:
        if student_interested:
            df_percentage.to_csv("data/{}_interested_null_percentage.csv".format(sheet_name))
        else:
            df_percentage.to_csv("data/{}_not_interested_null_percentage.csv".format(sheet_name))
            
    return df_percentage


## Interested Students

In [127]:
for sheet_name in SHEET_NAMES:
    build_var_null_percentage_df(df_interested, sheet_name, verbal = True, student_interested=True)

Pre & Post 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Self-Concept of Ability', 'abil_gen_bio1']
['Self-Concept of Ability', 'abil_gen_bio3']
['Self-Concept of Ability', 'abil_gen_chem1']
['Self-Concept of Ability', 'abil_gen_chem3']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,post_count,pre_count,original_null_count,post_null_count,pre_null_count,original_null_percentage,post_null_percentage,pre_null_percentage
0,aca1,Academic Integration,"Talk with faculty about academic matters, outs...",(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty ac...,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN
1,aca2,Academic Integration,Meet with an academic advisor concerning acade...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with academic a...,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN
2,aca3,Academic Integration,Meet with a student mentor concerning course a...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with student me...,NaN,70.0,NaN,NaN,29.0,NaN,NaN,0.292929,NaN
3,aca4,Academic Integration,Attend study groups outside of the classroom,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter attending study groups ...,NaN,69.0,NaN,NaN,30.0,NaN,NaN,0.303030,NaN
4,aca5,Academic Integration,Have informal or social contacts with faculty ...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty so...,NaN,69.0,NaN,NaN,30.0,NaN,NaN,0.303030,NaN


Weekly 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,w2_count,w3_count,w4_count,w5_count,original_null_count,w2_null_count,w3_null_count,w4_null_count,w5_null_count,original_null_percentage,w2_null_percentage,w3_null_percentage,w4_null_percentage,w5_null_percentage
0,att3,Attainment Value,How important to your identity is it to be kno...,"slider: 1=Not at all important, 7=Very important",important to my identity to know cooking science,NaN,80.0,71.0,67.0,65.0,NaN,19.0,28.0,32.0,34.0,NaN,0.191919,0.282828,0.323232,0.343434
1,badgradec,Grade Expectations - Course,Think about your grade in this course...what’s...,slider: 0-100%,worst course grade still satisfactory,NaN,82.0,72.0,69.0,67.0,NaN,17.0,27.0,30.0,32.0,NaN,0.171717,0.272727,0.303030,0.323232
2,badgradef,Grade Expectations - Final,Think about your grade on the final...what’s t...,slider: 0-100%,worst final grade still satisfactory,NaN,NaN,NaN,69.0,67.0,NaN,NaN,NaN,30.0,32.0,NaN,NaN,NaN,0.303030,0.323232
3,badgradem,Grade Expectations - Midterm,Think about your grade on the midterm...what’s...,slider: 0-100%,worst midterm grade still satisfactory,NaN,82.0,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,0.171717,NaN,NaN,NaN
4,badgradeq,Grade Expectations - Quiz,Think about your grade on this week's review q...,slider: 0-100%,worst quiz grade still satisfactory,NaN,82.0,72.0,69.0,67.0,NaN,17.0,27.0,30.0,32.0,NaN,0.171717,0.272727,0.303030,0.323232


Daily 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phonemodel']
['Study Participation', 'phonecarrier']




,Name Stem,Construct,Item,Response Values,Label,d1_count,d2_count,d3_count,d4_count,d5_count,d6_count,original_count,d1_null_count,d2_null_count,d3_null_count,d4_null_count,d5_null_count,d6_null_count,original_null_count,d1_null_percentage,d2_null_percentage,d3_null_percentage,d4_null_percentage,d5_null_percentage,d6_null_percentage,original_null_percentage
0,dadd,Reflection on Previous Day Activities,Was there additional course-related activity y...,"0=No, 1=Yes",additional course activity yesterday,NaN,62.0,63.0,62.0,61.0,47.0,NaN,NaN,37.0,36.0,37.0,38.0,52.0,NaN,NaN,0.373737,0.363636,0.373737,0.383838,0.525253,NaN
1,daddo,Reflection on Previous Day Activities,"If so, what else did you do?",open-ended,additional course activity - text,NaN,5.0,12.0,6.0,7.0,10.0,NaN,NaN,94.0,87.0,93.0,92.0,89.0,NaN,NaN,0.949495,0.878788,0.939394,0.929293,0.898990,NaN
2,dcact1,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 1 (day 1),NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,NaN,NaN,NaN,NaN,0.393939
3,dcact10,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 10 (day 2),NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,97.0,NaN,NaN,NaN,NaN,NaN,NaN,0.979798
4,dcact11,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 11 (day 3),NaN,NaN,NaN,NaN,NaN,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,0.424242


## All Students

In [128]:
for sheet_name in SHEET_NAMES:
    build_var_null_percentage_df(df, sheet_name, verbal = True, student_interested=False)

Pre & Post 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Self-Concept of Ability', 'abil_gen_bio1']
['Self-Concept of Ability', 'abil_gen_bio3']
['Self-Concept of Ability', 'abil_gen_chem1']
['Self-Concept of Ability', 'abil_gen_chem3']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,post_count,pre_count,original_null_count,post_null_count,pre_null_count,original_null_percentage,post_null_percentage,pre_null_percentage
0,aca1,Academic Integration,"Talk with faculty about academic matters, outs...",(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty ac...,NaN,100.0,NaN,NaN,69.0,NaN,NaN,0.408284,NaN
1,aca2,Academic Integration,Meet with an academic advisor concerning acade...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with academic a...,NaN,100.0,NaN,NaN,69.0,NaN,NaN,0.408284,NaN
2,aca3,Academic Integration,Meet with a student mentor concerning course a...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter meeting with student me...,NaN,100.0,NaN,NaN,69.0,NaN,NaN,0.408284,NaN
3,aca4,Academic Integration,Attend study groups outside of the classroom,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter attending study groups ...,NaN,99.0,NaN,NaN,70.0,NaN,NaN,0.414201,NaN
4,aca5,Academic Integration,Have informal or social contacts with faculty ...,(1) never ... (2) once a quarter ... (3) twice...,frequency this quarter talking with faculty so...,NaN,99.0,NaN,NaN,70.0,NaN,NaN,0.414201,NaN


Weekly 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phone']




,Name Stem,Construct,Item,Response Values,Label,original_count,w2_count,w3_count,w4_count,w5_count,original_null_count,w2_null_count,w3_null_count,w4_null_count,w5_null_count,original_null_percentage,w2_null_percentage,w3_null_percentage,w4_null_percentage,w5_null_percentage
0,att3,Attainment Value,How important to your identity is it to be kno...,"slider: 1=Not at all important, 7=Very important",important to my identity to know cooking science,NaN,103.0,91.0,83.0,80.0,NaN,66.0,78.0,86.0,89.0,NaN,0.390533,0.461538,0.508876,0.526627
1,badgradec,Grade Expectations - Course,Think about your grade in this course...what’s...,slider: 0-100%,worst course grade still satisfactory,NaN,106.0,92.0,85.0,82.0,NaN,63.0,77.0,84.0,87.0,NaN,0.372781,0.455621,0.497041,0.514793
2,badgradef,Grade Expectations - Final,Think about your grade on the final...what’s t...,slider: 0-100%,worst final grade still satisfactory,NaN,NaN,NaN,85.0,82.0,NaN,NaN,NaN,84.0,87.0,NaN,NaN,NaN,0.497041,0.514793
3,badgradem,Grade Expectations - Midterm,Think about your grade on the midterm...what’s...,slider: 0-100%,worst midterm grade still satisfactory,NaN,106.0,NaN,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN,0.372781,NaN,NaN,NaN
4,badgradeq,Grade Expectations - Quiz,Think about your grade on this week's review q...,slider: 0-100%,worst quiz grade still satisfactory,NaN,106.0,92.0,85.0,81.0,NaN,63.0,77.0,84.0,88.0,NaN,0.372781,0.455621,0.497041,0.520710


Daily 


Name Stems that are not in df:  
['Qualtrics Embedded Data', 'ip']
['Qualtrics Embedded Data', 'responseid']
['Qualtrics Embedded Data', 'recipientlastname']
['Qualtrics Embedded Data', 'recipientfirstname']
['Qualtrics Embedded Data', 'recipientemail']
['Student Entered Data', 'firstname']
['Student Entered Data', 'lastname']
['Student Entered Data', 'studentid']
['Student Entered Data', 'email']
['Study Participation', 'phonemodel']
['Study Participation', 'phonecarrier']




,Name Stem,Construct,Item,Response Values,Label,d1_count,d2_count,d3_count,d4_count,d5_count,d6_count,original_count,d1_null_count,d2_null_count,d3_null_count,d4_null_count,d5_null_count,d6_null_count,original_null_count,d1_null_percentage,d2_null_percentage,d3_null_percentage,d4_null_percentage,d5_null_percentage,d6_null_percentage,original_null_percentage
0,dadd,Reflection on Previous Day Activities,Was there additional course-related activity y...,"0=No, 1=Yes",additional course activity yesterday,NaN,72.0,77.0,77.0,75.0,56.0,NaN,NaN,97.0,92.0,92.0,94.0,113.0,NaN,NaN,0.573964,0.544379,0.544379,0.556213,0.668639,NaN
1,daddo,Reflection on Previous Day Activities,"If so, what else did you do?",open-ended,additional course activity - text,NaN,7.0,16.0,6.0,9.0,13.0,NaN,NaN,162.0,153.0,163.0,160.0,156.0,NaN,NaN,0.958580,0.905325,0.964497,0.946746,0.923077,NaN
2,dcact1,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 1 (day 1),NaN,NaN,NaN,NaN,NaN,NaN,77.0,NaN,NaN,NaN,NaN,NaN,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,0.544379
3,dcact10,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 10 (day 2),NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,167.0,NaN,NaN,NaN,NaN,NaN,NaN,0.988166
4,dcact11,Daily Course Activities,Please list all the course-related activities ...,open-ended,course activity 11 (day 3),NaN,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,104.0,NaN,NaN,NaN,NaN,NaN,NaN,0.615385


In [129]:
for c in df.columns:
    if "7" in c:
        print(c)

roster7_2
gr_lec7
gr_lec17
gr_readq7
gr_readq17
pre_oact7
pre_oact7rank
pre_oact7hrs
pre_orsh7
pre_er7x
pre_cost7
pre_iemot7
pre_agqmap7
pre_agqmap17
post_oactcompx7
post_oact7hrs
post_oact7rank
post_activ7
post_orsh7
post_er7x
post_cost7
post_iemot7
post_agqmap7
post_agqmap17
iemot7w2
woact7
woactrank7
woacthrs7
iemot7w3
woact17
woactrank17
woacthrs17
dcact7
doact7
dcactrank7
doactrank7
dcactimp7
dcactcomp7
dcactint7
dcactcost7
dcactchal7
dcactchce7
dcactreg7
dcactsat7
dcactcompx7
dcactimpx7
dcactintx7
dcactcostx7
dcact17
doact17
dcactrank17
doactrank17
dcactimp17
dcactcomp17
dcactint17
dcactcost17
dcactchal17
dcactchce17
dcactreg17
dcactsat17
dcactcompx17
dcactchalx17
dcactimpx17
dcactintx17
dcactcostx17
dcact27
doact27
dcactrank27
doactrank27
dcactimp27
dcactcomp27
dcactint27
dcactcost27
dcactchal27
dcactchce27
dcactreg27
dcactsat27
dcactcompx27
dcactchalx27
dcactimpx27
dcactintx27
dcactcostx27
iemot7w4
woact27
woactrank27
woacthrs27
iemot7w5
woact37
woactrank37
woacthrs37
